In [1]:
import copy 
import re

class FileManagerAFN():
    
    def __init__(self, file_name):
        self._file_name = file_name
        self._raw_data = None
        self._data = None
        
    def readFileAndConstructData(self):
        with open(self._file_name) as file:
            self._raw_data = file.readlines()
            self.removeEOF()
            self.separateData()
    
    def removeEOF(self):
        for i, data in enumerate(self.raw_data):
            if '\n' in data:
                index = data.find('\n')
                data = data[:index] + data[index+1:]
                self._raw_data[i] = data
                
    def separateData(self):
        list_of_lists = []
        for data in self._raw_data:
            data = data.split(',')
            list_of_lists.append(data)
        self._data = tuple(list_of_lists)
    
    def write_quintuple_in_file(self, quintuple):
        match = re.search(r".*(?=\.)", self._file_name)
        new_file_name = match.group() + "extended.txt"
        with open (new_file_name, "w") as file:
            self.write_states(file, quintuple)
            self.write_alphabet(file, quintuple)
            self.write_initial_state(file, quintuple)
            self.write_final_states(file, quintuple)
            self.write_transitions(file, quintuple)
        
    def write_states(self, file, quintuple):
        string_states = ""
        for state in quintuple.states:
            string_states += state.alphabet_character + ","
        string_states = string_states[:-1]
        string_states += "\n"
        file.write(string_states)
    
    def write_alphabet(self, file, quintuple):
        string_chars = ""
        for char in quintuple.alphabet:
            string_chars += char + ","
        string_chars = string_chars[:-1]
        string_chars += "\n"
        file.write(string_chars)
    
    def write_initial_state(self, file, quintuple):
        file.write(quintuple.initial_state.alphabet_character + "\n")
    
    def write_final_states(self, file, quintuple):
        string_states = ""
        for state in quintuple.states:
            if state.is_final:
                string_states += state.alphabet_character + ","
        string_states = string_states[:-1]
        string_states += "\n"
        file.write(string_states)
        
    def write_transitions(self, file, quintuple):
        for state in quintuple.states:
            for transition in state.transitions:
                string_transitions = ""
                string_transitions += "{},{},{}\n".format(state.alphabet_character,
                                                         transition.alphabet_transition,
                                                         transition.state_to.alphabet_character)
                file.write(string_transitions)
        
    @property
    def raw_data(self):
        return self._raw_data
    
    @property
    def data(self):
        return self._data
    
    

In [2]:
class Quintuple:
    
    def __init__(self, data):
        self._states = []
        self._alphabet = None
        self._initial_state = None
        self._final_states = []
        self.createQuintuple(data)

    def createQuintuple(self, data):
        self._alphabet = data[1]
        self.createStates(data)
        
    def createStates(self, data):
        for char_state in data[0]:
            is_initial = self.verifyIfContainsState(char_state, data[2])
            is_final = self.verifyIfContainsState(char_state, data[3])
            state = State(char_state, is_initial, is_final, None)
            self._states.append(state)
            self.addInitialOrfinal(state)
        self.createTransitionsPerState(data)
            
    
    def addInitialOrfinal(self, state):
        if state.is_initial:
            self._initial_state = state
        if state.is_final:
            self._final_states.append(state)
            
    def verifyIfContainsState(self, char, list_to_check):
        return True if (char in list_to_check) else False
        
    def createTransitionsPerState(self, data):
        for state in self._states:
            for i  in range(4, len(data)):
                if state.alphabet_character == data[i][0]:
                    transition = Transition(data[i][1], self.find_state_by_char(data[i][2]))
                    state.transitions.append(transition)
    
    def find_state_by_char(self, char_state):
        for state in self._states:
            if char_state in state.alphabet_character:
                return state
    
    def printQuintuple(self):
        print("Alphabet: ", self._alphabet)        
        print("The states are: ")
        self.printStates()
        print("\nInitial State: {}".format(self._initial_state.alphabet_character))
        print("Final State(s):")
        for state in self._final_states:
            print("{}".format(state.alphabet_character))
        
    def printStates(self):
        for state in self._states:
            state.printState()

            
    @property
    def states(self):
        return self._states
    
    @property
    def raw_data(self):
        return self._raw_data
    
    @property
    def initial_state(self):
        return self._initial_state
    
    @property
    def alphabet(self):
        return self._alphabet

In [3]:
class Transition():
    
    def __init__(self, alphabet_transition, state_to):
        self._alphabet_transition = alphabet_transition
        self._state_to = state_to
    
    @property
    def alphabet_transition(self):
        return self._alphabet_transition
    
    @alphabet_transition.setter
    def alphabet_transition(self, value):
        self._alphabet_transition = value
        
    @property
    def state_to(self):
        return self._state_to
    
    @state_to.setter
    def state_to(self, value):
        self._state_to = value

In [4]:
class State:
    
    def __init__(self, character, is_initial, is_final, transitions):
        self._alphabet_character = character
        self._is_initial = is_initial
        self._is_final = is_final
        self._transitions = []
        self._is_error_handler = False
        self._before_transition = None

    @property
    def alphabet_character(self):
        return self._alphabet_character
    
    @property
    def is_initial(self):
        return self._is_initial
    
    @property
    def is_final(self):
        return self._is_final
    
    @property
    def transitions(self):
        return self._transitions
    
    @property
    def is_error_handler(self):
        return self._is_error_handler
    
    @property
    def before_transition(self):
        return self._before_transition
    
    @before_transition.setter
    def before_transition(self, value):
        self._before_transition = value
    
    @alphabet_character.setter
    def alphabet_character(self, value):
        self._alphabet_character = value
    
    @is_error_handler.setter
    def is_error_handler(self, value):
        self._is_error_handler = value
        
    def printState(self):
        print("The State {}, is initial?: {}, is final?: {}, is error handler? {}, its transitions are:".format(
        self._alphabet_character, self._is_initial, self._is_final, self._is_error_handler))
        for index, transition in enumerate(self._transitions):
            print("{}-----{}----->{}".format(self._alphabet_character,
                                            transition.alphabet_transition,
                                            transition.state_to.alphabet_character))
        print("")

In [615]:
class ConvesionAFN():
    
    def __init__(self, quintuple):
        self._quintuple = quintuple
        
    def states_with_transitions(self):
        self._dict_states = dict()
        for character_alphabet in self.quintuple.alphabet:
            self.dict_states[character_alphabet] = []
        print(self.dict_states)
                
    def fill_states_transitions(self):
        for state in self.quintuple.states:
            for transition in state.transitions:
                for char_alphabet in self.quintuple.alphabet:
                    if char_alphabet == transition.alphabet_transition:
                        self.dict_states[char_alphabet].append(state)
                        
    def print_dict(self):
        for dict_key in self.dict_states:
            print(dict_key, end = ": ")
            for state  in self.dict_states[dict_key]:
                print(state.alphabet_character, end= ", ")
            print()
            
    def closure_epsilon(self, initial_state):
        cerradura_epsilon = []
        cerradura_epsilon.append(initial_state)
        self.iterate_closure(cerradura_epsilon[0], cerradura_epsilon)
        return cerradura_epsilon
                
    def iterate_closure(self, actual_state, cerradura_epsilon):
        for transition in actual_state.transitions:
            if transition.alphabet_transition == 'E':
                cerradura_epsilon.append(transition.state_to)
                self.iterate_closure(transition.state_to, cerradura_epsilon)
                
    def print_cerradura(self, cerradura_epsilon):
        for state in cerradura_epsilon:
            print(state.alphabet_character)
            
    def move_to(self, states, character):
        moves_to = []
        for dict_key in self.dict_states:
            if character == dict_key:
                for state in self.dict_states[dict_key]:
                    for state_s in states:
                        if state.alphabet_character == state_s.alphabet_character:
                            for transition in state.transitions:
                                if transition.alphabet_transition == character:
                                    moves_to.append(transition.state_to)
        return moves_to
    
    def print_moves(self, moves):
        for move in moves:
            print(move.alphabet_character)
            
    def go_to(self, states, character, actual_name): 
        """self.print_cerradura(states)
        print(character)
        print('-------------')"""
        
        
        moves_to = self.move_to(states, character)
        cerraduras = []
        for move in moves_to:
            cerradura = self.closure_epsilon(move)
            cerraduras += cerradura
        ##print(actual_name, ' ', character)
        if(len(cerraduras) > 0):
            self.remove_repeated(cerraduras)
            exists, name_in_dict = self.compare_before(cerraduras)
            if not actual_name in self.convertion_dict.keys():
                    self.convertion_dict[actual_name] = dict()
            if(exists):
                future_name = CreateName.getName()
                ##print(future_name, '-----')
                self.convertion_dict[actual_name][character] = future_name
                self.steps_dict[future_name] = cerraduras
                for dict_key in self.dict_states:
                    self.go_to(cerraduras, dict_key, future_name)
                
            else:
                self.convertion_dict[actual_name][character] = name_in_dict
  
        ##self.print_cerradura(cerraduras)
        ##return cerraduras
       
    
    def algorithm_conversion(self):
        self.steps_dict = dict()
        self.convertion_dict = dict()
        cerradura_initial = self.closure_epsilon(conversion_Afn.quintuple.initial_state)
        if(len(cerradura_initial) > 0):
            name = CreateName.getName()
            self.steps_dict[name] = cerradura_initial
            for dict_key in self.dict_states:
                self.go_to(cerradura_initial, dict_key, name)
                
    def compare_before(self, states):
        for dict_key in self.steps_dict:
            comparacion = self.steps_dict[dict_key]
            if len(comparacion) == len(states):
                comp = 0
                for i in range(len(comparacion)):
                    if comparacion[i].alphabet_character == states[i].alphabet_character:
                        comp+=1
                if(comp == len(comparacion)):
                    return False, dict_key 
        return True, None
    
    def print_steps(self):
        for dict_key in self.steps_dict:
            print(dict_key, end = ": ")
            for state  in self.steps_dict[dict_key]:
                print(state.alphabet_character, end= ", ")
            print()
        
    def print_table(self):
        for dict_key in self.convertion_dict:
            print(dict_key, ": ")
            for dict_key_1 in self.convertion_dict[dict_key]:
                print('\t',dict_key_1, end = ": ")
                print(self.convertion_dict[dict_key][dict_key_1])
            print()
    
    def remove_repeated(self, cerraduras):
        for index, state in enumerate(cerraduras):
            for index_compare, state_compare in enumerate(cerraduras):
                if index_compare != index:
                    if state.alphabet_character == state_compare.alphabet_character:
                        del cerraduras[index_compare]
                        print('iguales: ', state.alphabet_character, ' ', index_compare)
                
            
    @property
    def quintuple(self):
        return self._quintuple
    
    @quintuple.setter
    def quintuple(self, value):
        self._quintuple = value
        
    @property
    def dict_states(self):
        return self._dict_states
    
    @dict_states.setter
    def dict_states(self, value):
        self._dict_states = value

In [616]:
fileManager = FileManagerAFN("practice.txt")
fileManager.readFileAndConstructData()
print(fileManager.data)

(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16'], ['a', 'b'], ['0'], ['16'], ['0', 'E', '1'], ['0', 'E', '9'], ['1', 'E', '2'], ['1', 'E', '7'], ['2', 'E', '5'], ['2', 'E', '3'], ['3', 'b', '4'], ['4', 'E', '3'], ['4', 'E', '5'], ['5', 'a', '6'], ['6', 'E', '2'], ['6', 'E', '7'], ['7', 'b', '8'], ['8', 'E', '16'], ['9', 'E', '10'], ['9', 'E', '15'], ['10', 'E', '11'], ['11', 'a', '12'], ['12', 'E', '11'], ['12', 'E', '13'], ['13', 'b', '14'], ['14', 'E', '10'], ['14', 'E', '15'], ['15', 'E', '16'])


In [617]:
quintuple = Quintuple(fileManager.data)

In [618]:
quintuple.printQuintuple()

Alphabet:  ['a', 'b']
The states are: 
The State 0, is initial?: True, is final?: False, is error handler? False, its transitions are:
0-----E----->1
0-----E----->9

The State 1, is initial?: False, is final?: False, is error handler? False, its transitions are:
1-----E----->2
1-----E----->7

The State 2, is initial?: False, is final?: False, is error handler? False, its transitions are:
2-----E----->5
2-----E----->3

The State 3, is initial?: False, is final?: False, is error handler? False, its transitions are:
3-----b----->4

The State 4, is initial?: False, is final?: False, is error handler? False, its transitions are:
4-----E----->3
4-----E----->5

The State 5, is initial?: False, is final?: False, is error handler? False, its transitions are:
5-----a----->6

The State 6, is initial?: False, is final?: False, is error handler? False, its transitions are:
6-----E----->2
6-----E----->7

The State 7, is initial?: False, is final?: False, is error handler? False, its transitions are:

In [619]:
conversion_Afn = ConvesionAFN(quintuple)
conversion_Afn.states_with_transitions()
conversion_Afn.fill_states_transitions()
conversion_Afn.print_dict()

{'a': [], 'b': []}
a: 5, 11, 
b: 3, 7, 13, 


In [620]:
conversion_Afn.algorithm_conversion()

iguales:  16   9


In [621]:
conversion_Afn.print_steps()

B3: 0, 1, 2, 5, 3, 7, 9, 10, 11, 15, 16, 
C3: 6, 2, 5, 3, 7, 12, 11, 13, 
D3: 4, 3, 5, 8, 16, 14, 10, 11, 15, 
E3: 4, 3, 5, 
F3: 6, 2, 5, 3, 7, 
G3: 4, 3, 5, 8, 16, 


In [622]:
conversion_Afn.print_table()

B3 : 
	 a: C3
	 b: G3

C3 : 
	 a: C3
	 b: D3

D3 : 
	 a: C3
	 b: E3

E3 : 
	 a: F3
	 b: E3

F3 : 
	 a: F3
	 b: G3

G3 : 
	 a: F3
	 b: E3



In [498]:
a = dict()
a['a']

KeyError: 'a'

In [531]:
import string
class CreateName():
    
    returns  = 0
    counter = 0
    names = list(string.ascii_uppercase)
    
    @staticmethod
    def getName():
        if(len(CreateName.names) == CreateName.returns):
            CreateName.returns = 0
            CreateName.counter+=1
        if(CreateName.counter > 0):
            name = CreateName.names[CreateName.returns] + str(CreateName.counter)
        else:
            name = CreateName.names[CreateName.returns]
        CreateName.returns +=1
        return name

In [356]:
CreateName.getName()

'B'

In [5]:
class AFN():
    
    def __init__(self, quintuple):
        self._quintuple = quintuple
        self._debug = True
    
    def complete_quintuple(self, data):
        self.create_handle_error_state(data[0])
        index = 0
        for state in self._quintuple._states:
            index += 1
            if index <  len(self._quintuple._states):
                copy_alphabet = self._quintuple._alphabet.copy()
                for transition in state.transitions:
                    character = transition.alphabet_transition
                    self.remove_char_from_alphabet(character, copy_alphabet)
                print("mising in state {}: ".format(state.alphabet_character), copy_alphabet)
                self.update_transitions_new_state(copy_alphabet, self.quintuple.states[-1], state)
                
        
    def remove_char_from_alphabet(self, char, alphabet):
        try:
            index = alphabet.index(char)
            del alphabet[index]
        except:
            pass
    
    def create_handle_error_state(self, states):
        char_alphabet = str(max(list(map(lambda num: int(num), states))) + 1)
        state = State(char_alphabet, False, False, None)
        state.is_error_handler = True
        for alphabet in self._quintuple._alphabet:
            transition = Transition(alphabet, state)
            state.transitions.append(transition)
        self._quintuple._states.append(state)
        print("Created handle Error state: ")
        state.printState()
    
    def update_transitions_new_state(self, alphabet, state_to, state):
        for char in alphabet:
            transition = Transition(char, state_to)
            state.transitions.append(transition)
            
    def validate_string(self, string):
        self._is_valid_string = False
        self._list_paths = []
        self.has_state_transition(string, self._quintuple.initial_state)
        if not self._is_valid_string:
            print("Invalid String")
            
    
    def has_state_transition(self, string, state_actual):
        if self._debug:
            print("{}---q({})".format(string, state_actual.alphabet_character))
        if not string:
            return state_actual

        if self.in_alphabet(string[0]):
            for transition in state_actual.transitions:
                if string[0] in transition.alphabet_transition or transition.alphabet_transition == 'E':
                    copy_state = state_actual
                    if state_actual.is_initial:
                        copy_state = copy.deepcopy(state_actual)
                    copy_next_state = copy.deepcopy(transition.state_to)
                    self.append_path(copy_state, transition.alphabet_transition,  copy_next_state)
                    if transition._alphabet_transition == 'E':
                        final_state = self.has_state_transition(string, copy_next_state)
                    else: 
                        final_state = self.has_state_transition(string[1:], copy_next_state)
                    if final_state is not None and  not final_state.is_error_handler:
                        self.browse_until_final(final_state)
        else:
            if string:
                copy_state = state_actual
                if state_actual.is_initial:
                    copy_state = copy.deepcopy(state_actual)
                copy_next_state = copy.deepcopy(copy_state)
                self.append_exception_handled(copy_state, string[0], copy_next_state)
                final_state = self.has_state_transition(string[1:], copy_next_state)
                if final_state is not None and  not final_state.is_error_handler:
                        self.browse_until_final(final_state)
            
    def in_alphabet(self, char):
        for char_alphabet  in self.quintuple.alphabet:
            if char == char_alphabet:
                return True
        return False
    
    def append_path(self, state_actual_copy, alphabet_transition, next_state):
        if state_actual_copy.before_transition is not None:
            string = "({}) ---> q{}".format(alphabet_transition,
                                            next_state.alphabet_character)
            next_state.before_transition = state_actual_copy.before_transition + string
        else:
            string = "q{}({}) ---> q{}".format(state_actual_copy.alphabet_character, 
                                            alphabet_transition,
                                            next_state.alphabet_character)
            next_state.before_transition = string
    
    def append_exception_handled(self, actual_state, character, next_state):
        string = " Exception handled in state {}, character removed: {} ---> \
q{}".format(actual_state.alphabet_character,character, actual_state.alphabet_character)
        if actual_state.before_transition is not None:
            next_state.before_transition = actual_state.before_transition + string
        else:
            next_state.before_transition = string
            
    
    def browse_until_final(self, final_state):
        if final_state.is_final:
            self._is_valid_string = True
            return print("The String is success with the next path:\n",
                         final_state.before_transition, "\n")
        for transition in final_state.transitions:
            if transition.alphabet_transition == "E":
                copy_next_state = copy.deepcopy(transition.state_to)
                self.append_path(final_state, transition.alphabet_transition, copy_next_state)
                self.browse_until_final(copy_next_state)
                
                                      
    def print_stack_trace(self, final_state):
        self._is_valid_string = True
        actual_state = final_state
        string = " q{}({}) ---> q{}".format(actual_state.before_transition.state_to.alphabet_character, 
                                                actual_state.before_transition.alphabet_transition,
                                                actual_state.alphabet_character)
        actual_state = actual_state.before_transition.state_to
        print("The String is success and was completed with the next path")
        while actual_state.before_transition is not None:
            string = " q{}({}) ---> ".format(actual_state.before_transition.state_to.alphabet_character, 
                                                actual_state.before_transition.alphabet_transition) + string
            actual_state = actual_state.before_transition.state_to
        print(string)
    
            
    @property
    def quintuple(self):
        return self._quintuple
    
    @property
    def debug(self):
        return self._debug
    
    @debug.setter
    def debug(self, value):
        self._debug = value

In [ ]:
afn = AFN(quintuple)

In [ ]:
afn.complete_quintuple(fileManager.data)
fileManager.write_quintuple_in_file(afn.quintuple)

In [ ]:
afn.quintuple.printQuintuple()

In [ ]:
afn.debug = False
afn.validate_string("ababab")

In [ ]:
afn.debug = False
afn.validate_string("@abca_b@ab@")

In [ ]:
afn.validate_string("@abca_b@ab@")

# Cambiar por maps y labdas